In [3]:
import caiman as cm
from caiman.motion_correction import MotionCorrect
from caiman.source_extraction.cnmf import cnmf as cnmf
from caiman.source_extraction.cnmf import params as params
from glob import glob
import numpy as np
from caiman.utils.visualization import nb_plot_contour
import os
from glob import glob
import numpy as np
import scipy.io as sio
import time
import sys

sys.path.append('C:/Users/Will/Lab Code/sameCells')
import online_analysis as oa

In [86]:
fr = 30  # imaging rate in frames per second
dxy = (1.5, 1.5, 1.5)  # spatial resolution in x and y in (um per pixel)
max_shift_um = (12., 12., 12.) # maximum shift in um
patch_motion_um = (100., 100., 100.)  # patch size for non-rigid correction in um
max_shifts = [int(a/b) for a, b in zip(max_shift_um, dxy)]
strides = tuple([int(a/b) for a, b in zip(patch_motion_um, dxy)])
overlaps = (24, 24, 24)
max_deviation_rigid = 3

pw_rigid = False      # flag to select rigid vs pw_rigid motion correction, False

mc_dict = {
   'dxy': dxy,
   'pw_rigid': pw_rigid,
   'max_shifts': max_shifts,
   'strides': strides,
   'overlaps': overlaps,
   'max_deviation_rigid': max_deviation_rigid,
   'border_nan': 'copy',
   'is3D': True
}

opts = params.CNMFParams(params_dict=mc_dict)

rf = None  # half-size of the patches in pixels. Should be `None` when seeded CNMF is used.
only_init = False   # has to be `False` when seeded CNMF is used
gSig = (5, 5, 5)    # expected half size of neurons in pixels
motion_correct = True

opts_dict = {
    'fr': fr,
    'decay_time': 1.0,
    'p': 2, # deconv 0 is off, 1 is slow, 2 is fast
    'nb': 2, # background compenents -> nb = 3 for complex
    'rf': rf,
    'only_init': only_init,
    'gSig': gSig,
    'ssub': 2,
    'tsub': 2,
    'merge_thr': 0.85,
    'motion_correct': motion_correct
}

opts.change_params(opts_dict);

In [82]:
#%% start the cluster (if a cluster already exists terminate it)
if 'dview' in locals():
    cm.stop_server(dview=dview)
c, dview, n_processes = cm.cluster.setup_cluster(
    backend='local', n_processes=None, single_thread=False)

In [47]:
from ScanImageTiffReader import ScanImageTiffReader
import matplotlib.pyplot as plt

In [65]:
with ScanImageTiffReader(fnames[0]) as reader:
    data = reader.data()[slice(0,-1,2),:,100:412] # remove red channel
data = data.flatten().reshape(150, 512, 312, 3) # morhp into T x X x Y x Z

In [74]:
mm = cm.save_memmap([data], base_name='M3D', order='C')

In [69]:
# mc = MotionCorrect([mm], **opts.get_group('motion'))

In [83]:
Yr, dims, T = cm.load_memmap(mm)
images = np.reshape(Yr.T, [T] + list(dims), order='F')

In [84]:
images.shape

(150, 512, 312, 3)

(150, 512, 312, 3)

In [87]:
c = cnmf.CNMF(n_processes=n_processes, params=opts)
c.fit(images)

IndexError: index 2 is out of bounds for axis 0 with size 2

In [31]:
fnames = [glob('E:/caiman tests/stimtest2/*.tif')[0]]
memmap = cm.save_memmap(fnames, base_name=f'MAP{0+1}', order='F',
                        slices=[slice(0,-1,2),
                                slice(0,512),
                                slice(100,400)])

In [33]:
folder_mmaps = glob('E:/caiman tests/stimtest2/MAP*.mmap')
folder_mmaps

['E:/caiman tests/stimtest2\\MAP1_d1_512_d2_300_d3_1_order_F_frames_450_.mmap']

In [39]:
Yr, dims, T = cm.load_memmap(memmap)
Y = np.reshape(Yr, dims + (T,), order='F')

In [40]:
Y.shape

(512, 300, 450)

In [25]:
mc = MotionCorrect(folder_mmaps, **opts.get_group('motion'))

In [27]:
mc.motion_correct(template=mc.total_template_rig)

ValueError: assignment destination is read-only